FB-22, Venher Pavlo, lab 4, part 1

Imports

In [6]:
import pandas as pd
import numpy as np
import timeit as ti

Prepare data

In [11]:
data_df = pd.read_csv('dbs/household_electric.csv', na_values='?', dtype={'Global_active_power': float, 'Global_reactive_power': float, 'Voltage': float, 'Global_intensity': float, 'Sub_metering_1': float, 'Sub_metering_2': float, 'Sub_metering_3': float})
data_df = data_df.dropna()
data_df['DateTime'] = pd.to_datetime(data_df['Date'] + ' ' + data_df['Time'], dayfirst=True)

for column in data_df.columns[2:-1]:
    data_df[column] = pd.to_numeric(data_df[column], errors='coerce')

data_np = data_df.to_numpy()

1

In [14]:
def part1np(data):
    return data[data[:, 2] > 5]

def part2df(data):
    return data[data['Global_active_power'] > 5]

# print('np:\n', task1_np(data_np)[:, [0, 1, 2]])
print("Time np:", ti.timeit(lambda: part1np(data_np), number=1))
# print('df:\n', task1_df(data_df)[['Date', 'Time', 'Global_active_power']])
print("Time df:", ti.timeit(lambda: part2df(data_df), number=1))

Time np: 0.04048590001184493
Time df: 0.00561829999787733


2

In [17]:
def part2np(data):
    return data[data[:, 4] > 235]

def part2df(data):
    return data[data['Voltage'] > 235]

# print('np:\n', part2np(data_np)[:, [0, 1, 4]])
print("Time np:", ti.timeit(lambda: part2np(data_np), number=1))
# print('df:\n', part2df(data_df)[['Date', 'Time', 'Voltage']])
print("Time df:", ti.timeit(lambda: part2df(data_df), number=1))

Time np: 0.2004750000196509
Time df: 0.07971069996710867


3

In [20]:
def part3np(data):
    return data[(data[:, 5] >= 19) & (data[:, 5] <= 20) & (data[:, 7] > data[:, 8])]

def part3df(data):
    return data[(data['Global_intensity'] >= 19) & (data['Global_intensity'] <= 20) & (data['Sub_metering_2'] > data['Sub_metering_3'])]

# print('np:\n', task3_np(data_np)[:, [0, 1, 5, 7, 8]])
print("Time np:", ti.timeit(lambda: part3np(data_np), number=1))
# print('df:\n', task3_df(data_df)[['Date', 'Time', 'Global_intensity', 'Sub_metering_2', 'Sub_metering_3']])
print("Time df:", ti.timeit(lambda: part3df(data_df), number=1))

Execution time with numpy array: 0.10584259999450296
Execution time with DataFrame: 0.014366200019139796


4

In [21]:
def part4np(data):
    random_sample_np = data[np.random.choice(data_np.shape[0], size=500000, replace=False), :]
    average_consumption = np.mean(random_sample_np[:, 6:9], axis=0)
    return average_consumption

def part4df(data):
    random_sample_df = data.sample(n=500000, replace=False)
    average_consumption = random_sample_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
    return average_consumption

# print('np:\n', part4np(data_np))
print("Time np:", ti.timeit(lambda: part4np(data_np), number=1))
# print('df:\n', part4df(data_df))
print("Time df:", ti.timeit(lambda: part4df(data_df), number=1))

Execution time with numpy array: 0.3833242999971844
Execution time with DataFrame: 0.13659930002177134


5

In [23]:
def part5np(data):
    hours = np.array([int(time.split(':')[0]) for time in data[:, 1]])
    after_18 = data[hours >= 18]
    high_power_consumption = after_18[after_18[:, 2].astype(float) > 6]
    filtered_households = high_power_consumption[
        (high_power_consumption[:, 6].astype(float) + 
         high_power_consumption[:, 7].astype(float) + 
         high_power_consumption[:, 8].astype(float)) > 6]
    first_half = filtered_households[:len(filtered_households) // 2]
    second_half = filtered_households[len(filtered_households) // 2:]
    result = np.concatenate([first_half[::3], second_half[::4]])
    return result

def part5df(data):
    df_after_18 = data[data['DateTime'].dt.hour >= 18]
    average_power_consumption = df_after_18.groupby('DateTime')['Global_active_power'].mean()
    high_power_consumption = average_power_consumption[average_power_consumption > 6]
    filtered_households = data[data['DateTime'].isin(high_power_consumption.index)]
    filtered_households = filtered_households[(filtered_households['Sub_metering_1'] + filtered_households['Sub_metering_2'] + filtered_households['Sub_metering_3']) > 6]
    first_half = filtered_households.iloc[:len(filtered_households)//2]
    second_half = filtered_households.iloc[len(filtered_households)//2:]
    result = pd.concat([first_half.iloc[::3], second_half.iloc[::4]])
    return result

# print('np:\n', part5np(data_np))
print("Time np:", ti.timeit(lambda: part5np(data_np), number=1))
# print('df:\n', part5df(data_df))
print("Time df:", ti.timeit(lambda: part5df(data_df), number=1))

Time np: 0.4773177999886684
Time df: 0.15220399998361245
